In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.cm as cm

# --- Given parameters ---
Lambda = 10.25
beta = 0.0004
mu = 0.02
alpha0 = 0.0051
alpha1 = 0.021
alpha2 = 0.03
b = 0.0083
gamma = 0.006
delta = 0.04

# --- Define the basic reproduction number R0 ---
def calculate_R0(beta, alpha1, gamma, mu):
    return (2 * beta) / ( alpha1 + gamma + mu)

# --- Define Sensitivity Functions ---
def calculate_SEN_beta(beta, alpha1, gamma, mu):
    R0 = calculate_R0(beta, alpha1, gamma, mu)
    d_R0_d_beta = 2 / (alpha1 + gamma + mu)
    return (beta / R0) * d_R0_d_beta


def calculate_SEN_alpha1(beta, alpha1, gamma, mu):
    R0 = calculate_R0(beta, alpha1, gamma, mu)
    d_R0_d_alpha1 = - (2 * beta) / ((alpha1 + gamma + mu)**2)
    return (alpha1 / R0) * d_R0_d_alpha1


def calculate_SEN_gamma(beta, alpha1, gamma, mu):
    R0 = calculate_R0(beta, alpha1, gamma, mu)
    d_R0_d_gamma = - (2 * beta) / ((alpha1 + gamma + mu)**2)
    return (gamma / R0) * d_R0_d_gamma

def calculate_SEN_mu(beta, alpha1, gamma, mu):
    R0 = calculate_R0(beta, alpha1, gamma, mu)
    d_R0_d_mu = - (2 * beta) / ((alpha1 + gamma + mu)**2)
    return (mu / R0) * d_R0_d_mu

# --- Parameter Ranges (using smaller values for better visualization) ---
beta_vals = np.linspace(0.0003, 0.0005, 50)
alpha1_vals = np.linspace(0.020, 0.022, 50)
gamma_vals = np.linspace(0.005, 0.007, 50)
mu_vals = np.linspace(0.019, 0.021, 50)

# --- Define PDF File ---
pdf_pages = PdfPages('SEN_contour_plots_ALL_new.pdf') # Output to PDF - try a new name

# --- Parameter Pairs and Axis Indices (All Possible Combinations) ---
parameters = {'beta': beta, 'alpha1': alpha1, 'gamma': gamma, 'mu': mu}
parameter_names = list(parameters.keys())
latex_names = {'beta': r'\beta', 'alpha1': r'\alpha_1', 'gamma': r'\gamma', 'mu': r'\mu'} # ADDED
sensitivity_functions = {
    'beta': calculate_SEN_beta,
    'alpha1': calculate_SEN_alpha1,
    'gamma': calculate_SEN_gamma,
    'mu': calculate_SEN_mu
}

# Create the figure and subplots - OUTSIDE the loops!
fig, axes = plt.subplots(5, 3, figsize=(20, 20))
axes = axes.flatten()

plot_index = 0
for i in range(len(parameter_names)):
    for j in range(i + 1, len(parameter_names)): # Ensure no repeat and i<j

        # --- Plot ---
        ax = axes[plot_index] # Get correct axes from axes array
        # ---
        x_name = parameter_names[i]
        y_name = parameter_names[j]
        x_latex = latex_names[x_name] # ADDED
        y_latex = latex_names[y_name] # ADDED

        # Create values and meshgrid
        x_vals = np.linspace(parameters[x_name]*0.8, parameters[x_name]*1.2, 50) # Example range
        y_vals = np.linspace(parameters[y_name]*0.8, parameters[y_name]*1.2, 50) # Example Range
        X, Y = np.meshgrid(x_vals, y_vals)

        # Calculate combined sensitivity
        SEN_values = np.zeros_like(X)
        for row in range(X.shape[0]):
            for col in range(X.shape[1]):
                # Updated Parameters for this spot in X,Y matrix
                updated_params = parameters.copy()
                updated_params[x_name] = X[row, col]
                updated_params[y_name] = Y[row, col]

                #Sensitivity at the given x and y value in the matrix
                SEN_x = sensitivity_functions[x_name](updated_params['beta'], updated_params['alpha1'], updated_params['gamma'], updated_params['mu'])
                SEN_y = sensitivity_functions[y_name](updated_params['beta'], updated_params['alpha1'], updated_params['gamma'], updated_params['mu'])
                SEN_values[row, col] = SEN_x * SEN_y

        # Plot Contour
        contour = ax.contourf(X, Y, SEN_values, levels=20, cmap='coolwarm')
        fig.colorbar(contour, ax=ax, label=r'$\frac{\partial R_0}{\partial ' + x_latex + r'}\cdot \frac{\partial R_0}{\partial ' + y_latex + r'}$', shrink=0.7) #MODIFIED

        ax.set_xlabel(r'$' + x_latex + r'$') #MODIFIED
        ax.set_ylabel(r'$' + y_latex + r'$') #MODIFIED
        ax.set_title(r'SEN($' + x_latex + r'$) vs SEN($' + y_latex + r'$)') #MODIFIED

        plot_index += 1

# Remove any unused subplots
for i in range(plot_index, len(axes)):
    fig.delaxes(axes[i])

# Adjust layout and save
plt.tight_layout()
pdf_pages.savefig(fig)
plt.close(fig)

# --- Close PDF File ---
pdf_pages.close() # Close the PDF file.  Important!
print('Sensitivity contour plots generated and saved to SEN_contour_plots_ALL.pdf')

Sensitivity contour plots generated and saved to SEN_contour_plots_ALL.pdf


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
zeta_values = [0.3, 0.35, 0.4, 0.45]
varsigma_end = 1000
p = 10
Lambda = 10.25
beta = 0.0004
mu = 0.02
alpha0 = 0.0051
alpha1 = 0.021
b = 0.0083
gamma_val = 0.006
delta = 0.04
h = 0.01
psi = 2 #set psi here

# Time span
varsigma = np.arange(0, varsigma_end, h)
M = len(varsigma)

# Initial conditions
S0 = 100
I0 = 50
R0 = 60

# Define w(varsigma)
def w(varsigma_val):
    return varsigma_val + 1  # Correct definition

# Define functions for p_1, p_2, and p_3
def p1(varsigma_m, S, I, R):
    return Lambda - (2 * beta * S * I) / (S + I + R) - mu * S + delta * R

def p2(varsigma_m, S, I, R):
    return (2 * beta * S * I) / (S + I + R) - (alpha0 + alpha1 * (b / (b + I))) * I - (gamma_val + mu) * I

def p3(varsigma_m, S, I, R):
    return (alpha0 + alpha1 * (b / (b + I))) * I - (mu + delta) * R

# Define A and B
def A(m, l, psi):
    return (m - l)**psi * (m - l + 1 + psi) - (m - l + 1)**(psi + 1)

def B(m, l, psi):
    return (m - l + 1)**psi * (m - l + 2 + psi) - (m - l)**psi * (m - l + 2 + 2 * psi)

def fractional_sir_solver(zeta, S0, I0, R0):
    S = np.zeros(M)
    I = np.zeros(M)
    R = np.zeros(M)
    S[0] = S0
    I[0] = I0
    R[0] = R0

    for m in range(M - 1):
        sum_S = 0
        sum_I = 0
        sum_R = 0

        for l in range(m + 1):
            if l == 0:
                sum_S += p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
            else:
                sum_S += p1(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)

        S[m + 1] = (w(0) / w(varsigma[m])) * S0 + zeta * p1(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_S
        I[m + 1] = (w(0) / w(varsigma[m])) * I0 + zeta * p2(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_I
        R[m + 1] = (w(0) / w(varsigma[m])) * R0 + zeta * p3(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_R
    return S, I, R

# --- Plotting and PDF Saving ---
pdf_filenames = ["susceptible.pdf", "infected.pdf", "recovered.pdf"]
titles = [
    "Susceptible Population (S) with Different Fractional Orders",
    "Infected Population (I) with Different Fractional Orders",
    "Recovered Population (R) with Different Fractional Orders",
]
ylabels = ["S", "I", "R"]
data_labels = ["S", "I", "R"]

for pdf_filename, title, ylabel, data_label in zip(pdf_filenames, titles, ylabels, data_labels):
    with PdfPages(pdf_filename) as pdf:
      
        # Lists to hold all data
        all_data = []
        
        # Collect data
        for zeta in zeta_values:
          S, I, R = fractional_sir_solver(zeta, S0, I0, R0)
          if data_label == "S":
             all_data.append(S)
          elif data_label == "I":
            all_data.append(I)
          elif data_label =="R":
             all_data.append(R)
      
        # Calculate mins and maxs
        data_min = min([min(data) for data in all_data])
        data_max = max([max(data) for data in all_data])


        # Create a figure
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)
      
        # Plot data
        for zeta, data in zip(zeta_values, all_data):
           ax.plot(varsigma, data, label=f'ζ={zeta}')
        
        ax.set_xlabel('ς')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.set_ylim(data_min, data_max)
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Plots saved to {pdf_filenames}")

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
zeta_values = [0.3, 0.35, 0.4, 0.45]
varsigma_end = 1000
p = np.e
Lambda = 10.25
beta = 0.0004
mu = 0.02
alpha0 = 0.0051
alpha1 = 0.021
b = 0.0083
gamma_val = 0.006
delta = 0.04
h = 0.01
psi = 1 #set psi here

# Time span
varsigma = np.arange(0, varsigma_end, h)
M = len(varsigma)

# Initial conditions
S0 = 100
I0 = 50
R0 = 60

# Define w(varsigma)
def w(varsigma_val):
    return 1  # Correct definition

# Define functions for p_1, p_2, and p_3
def p1(varsigma_m, S, I, R):
    return Lambda - (2 * beta * S * I) / (S + I + R) - mu * S + delta * R

def p2(varsigma_m, S, I, R):
    return (2 * beta * S * I) / (S + I + R) - (alpha0 + alpha1 * (b / (b + I))) * I - (gamma_val + mu) * I

def p3(varsigma_m, S, I, R):
    return (alpha0 + alpha1 * (b / (b + I))) * I - (mu + delta) * R

# Define A and B
def A(m, l, psi):
    return (m - l)**psi * (m - l + 1 + psi) - (m - l + 1)**(psi + 1)

def B(m, l, psi):
    return (m - l + 1)**psi * (m - l + 2 + psi) - (m - l)**psi * (m - l + 2 + 2 * psi)

def fractional_sir_solver(zeta, S0, I0, R0):
    S = np.zeros(M)
    I = np.zeros(M)
    R = np.zeros(M)
    S[0] = S0
    I[0] = I0
    R[0] = R0

    for m in range(M - 1):
        sum_S = 0
        sum_I = 0
        sum_R = 0

        for l in range(m + 1):
            if l == 0:
                sum_S += p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
            else:
                sum_S += p1(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)

        S[m + 1] = (w(0) / w(varsigma[m])) * S0 + zeta * p1(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_S
        I[m + 1] = (w(0) / w(varsigma[m])) * I0 + zeta * p2(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_I
        R[m + 1] = (w(0) / w(varsigma[m])) * R0 + zeta * p3(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_R
    return S, I, R

# --- Plotting and PDF Saving ---
pdf_filenames = ["susceptible.pdf", "infected.pdf", "recovered.pdf"]
titles = [
    "Susceptible Population (S) with Different Fractional Orders",
    "Infected Population (I) with Different Fractional Orders",
    "Recovered Population (R) with Different Fractional Orders",
]
ylabels = ["S", "I", "R"]
data_labels = ["S", "I", "R"]

for pdf_filename, title, ylabel, data_label in zip(pdf_filenames, titles, ylabels, data_labels):
    with PdfPages(pdf_filename) as pdf:
      
        # Lists to hold all data
        all_data = []
        
        # Collect data
        for zeta in zeta_values:
          S, I, R = fractional_sir_solver(zeta, S0, I0, R0)
          if data_label == "S":
             all_data.append(S)
          elif data_label == "I":
            all_data.append(I)
          elif data_label =="R":
             all_data.append(R)
      
        # Calculate mins and maxs
        data_min = min([min(data) for data in all_data])
        data_max = max([max(data) for data in all_data])


        # Create a figure
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)
      
        # Plot data
        for zeta, data in zip(zeta_values, all_data):
           ax.plot(varsigma, data, label=f'ζ={zeta}')
        
        ax.set_xlabel('ς')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.set_ylim(data_min, data_max)
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Plots saved to {pdf_filenames}")

Plots saved to ['susceptible.pdf', 'infected.pdf', 'recovered.pdf']


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
zeta_values = [0.3, 0.35, 0.4, 0.45]
varsigma_end = 1000
p = np.e
Lambda = 10.25
beta = 0.0004
mu = 0.02
alpha0 = 0.0051
alpha1 = 0.021
b = 0.0083
gamma_val = 0.006
delta = 0.04
h = 0.01
psi = zeta #set psi here

# Time span
varsigma = np.arange(0, varsigma_end, h)
M = len(varsigma)

# Initial conditions
S0 = 100
I0 = 50
R0 = 60

# Define w(varsigma)
def w(varsigma_val):
    return 1  # Correct definition

# Define functions for p_1, p_2, and p_3
def p1(varsigma_m, S, I, R):
    return Lambda - (2 * beta * S * I) / (S + I + R) - mu * S + delta * R

def p2(varsigma_m, S, I, R):
    return (2 * beta * S * I) / (S + I + R) - (alpha0 + alpha1 * (b / (b + I))) * I - (gamma_val + mu) * I

def p3(varsigma_m, S, I, R):
    return (alpha0 + alpha1 * (b / (b + I))) * I - (mu + delta) * R

# Define A and B
def A(m, l, psi):
    return (m - l)**psi * (m - l + 1 + psi) - (m - l + 1)**(psi + 1)

def B(m, l, psi):
    return (m - l + 1)**psi * (m - l + 2 + psi) - (m - l)**psi * (m - l + 2 + 2 * psi)

def fractional_sir_solver(zeta, S0, I0, R0):
    S = np.zeros(M)
    I = np.zeros(M)
    R = np.zeros(M)
    S[0] = S0
    I[0] = I0
    R[0] = R0

    for m in range(M - 1):
        sum_S = 0
        sum_I = 0
        sum_R = 0

        for l in range(m + 1):
            if l == 0:
                sum_S += p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
            else:
                sum_S += p1(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)

        S[m + 1] = (w(0) / w(varsigma[m])) * S0 + zeta * p1(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_S
        I[m + 1] = (w(0) / w(varsigma[m])) * I0 + zeta * p2(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_I
        R[m + 1] = (w(0) / w(varsigma[m])) * R0 + zeta * p3(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_R
    return S, I, R

# --- Plotting and PDF Saving ---
pdf_filenames = ["susceptible.pdf", "infected.pdf", "recovered.pdf"]
titles = [
    "Susceptible Population (S) with Different Fractional Orders",
    "Infected Population (I) with Different Fractional Orders",
    "Recovered Population (R) with Different Fractional Orders",
]
ylabels = ["S", "I", "R"]
data_labels = ["S", "I", "R"]

for pdf_filename, title, ylabel, data_label in zip(pdf_filenames, titles, ylabels, data_labels):
    with PdfPages(pdf_filename) as pdf:
      
        # Lists to hold all data
        all_data = []
        
        # Collect data
        for zeta in zeta_values:
          S, I, R = fractional_sir_solver(zeta, S0, I0, R0)
          if data_label == "S":
             all_data.append(S)
          elif data_label == "I":
            all_data.append(I)
          elif data_label =="R":
             all_data.append(R)
      
        # Calculate mins and maxs
        data_min = min([min(data) for data in all_data])
        data_max = max([max(data) for data in all_data])


        # Create a figure
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)
      
        # Plot data
        for zeta, data in zip(zeta_values, all_data):
           ax.plot(varsigma, data, label=f'ζ={zeta}')
        
        ax.set_xlabel('ς')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.set_ylim(data_min, data_max)
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Plots saved to {pdf_filenames}")

Plots saved to ['susceptible.pdf', 'infected.pdf', 'recovered.pdf']


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
zeta_values = [0.3, 0.35, 0.4, 0.45]
varsigma_end = 1000
p = np.e
Lambda = 10.25
beta = 0.0004
mu = 0.02
alpha0 = 0.0051
alpha1 = 0.021
b = 0.0083
gamma_val = 0.006
delta = 0.04
h = 0.01
psi = 2 #set psi here

# Time span
varsigma = np.arange(0, varsigma_end, h)
M = len(varsigma)

# Initial conditions
S0 = 100
I0 = 50
R0 = 60

# Define w(varsigma)
def w(varsigma_val):
    return varsigma_val + 2  # Correct definition

# Define functions for p_1, p_2, and p_3
def p1(varsigma_m, S, I, R):
    return Lambda - (2 * beta * S * I) / (S + I + R) - mu * S + delta * R

def p2(varsigma_m, S, I, R):
    return (2 * beta * S * I) / (S + I + R) - (alpha0 + alpha1 * (b / (b + I))) * I - (gamma_val + mu) * I

def p3(varsigma_m, S, I, R):
    return (alpha0 + alpha1 * (b / (b + I))) * I - (mu + delta) * R

# Define A and B
def A(m, l, psi):
    return (m - l)**psi * (m - l + 1 + psi) - (m - l + 1)**(psi + 1)

def B(m, l, psi):
    return (m - l + 1)**psi * (m - l + 2 + psi) - (m - l)**psi * (m - l + 2 + 2 * psi)

def fractional_sir_solver(zeta, S0, I0, R0):
    S = np.zeros(M)
    I = np.zeros(M)
    R = np.zeros(M)
    S[0] = S0
    I[0] = I0
    R[0] = R0

    for m in range(M - 1):
        sum_S = 0
        sum_I = 0
        sum_R = 0

        for l in range(m + 1):
            if l == 0:
                sum_S += p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
            else:
                sum_S += p1(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)

        S[m + 1] = (w(0) / w(varsigma[m])) * S0 + zeta * p1(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_S
        I[m + 1] = (w(0) / w(varsigma[m])) * I0 + zeta * p2(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_I
        R[m + 1] = (w(0) / w(varsigma[m])) * R0 + zeta * p3(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_R
    return S, I, R

# --- Plotting and PDF Saving ---
pdf_filenames = ["susceptible.pdf", "infected.pdf", "recovered.pdf"]
titles = [
    "Susceptible Population (S) with Different Fractional Orders",
    "Infected Population (I) with Different Fractional Orders",
    "Recovered Population (R) with Different Fractional Orders",
]
ylabels = ["S", "I", "R"]
data_labels = ["S", "I", "R"]

for pdf_filename, title, ylabel, data_label in zip(pdf_filenames, titles, ylabels, data_labels):
    with PdfPages(pdf_filename) as pdf:
      
        # Lists to hold all data
        all_data = []
        
        # Collect data
        for zeta in zeta_values:
          S, I, R = fractional_sir_solver(zeta, S0, I0, R0)
          if data_label == "S":
             all_data.append(S)
          elif data_label == "I":
            all_data.append(I)
          elif data_label =="R":
             all_data.append(R)
      
        # Calculate mins and maxs
        data_min = min([min(data) for data in all_data])
        data_max = max([max(data) for data in all_data])


        # Create a figure
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)
      
        # Plot data
        for zeta, data in zip(zeta_values, all_data):
           ax.plot(varsigma, data, label=f'ζ={zeta}')
        
        ax.set_xlabel('ς')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.set_ylim(data_min, data_max)
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Plots saved to {pdf_filenames}")

Plots saved to ['susceptible.pdf', 'infected.pdf', 'recovered.pdf']


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
zeta_values = [0.8, 0.85, 0.9, 0.95]
varsigma_end = 1000
p = np.e
Lambda = 10.25
beta = 0.0004
mu = 0.02
alpha0 = 0.0051
alpha1 = 0.021
b = 0.0083
gamma_val = 0.006
delta = 0.04
h = 0.01
psi = 2 #set psi here

# Time span
varsigma = np.arange(0, varsigma_end, h)
M = len(varsigma)

# Initial conditions
S0 = 100
I0 = 50
R0 = 60

# Define w(varsigma)
def w(varsigma_val):
    return varsigma_val  # Correct definition

# Define functions for p_1, p_2, and p_3
def p1(varsigma_m, S, I, R):
    return Lambda - (2 * beta * S * I) / (S + I + R) - mu * S + delta * R

def p2(varsigma_m, S, I, R):
    return (2 * beta * S * I) / (S + I + R) - (alpha0 + alpha1 * (b / (b + I))) * I - (gamma_val + mu) * I

def p3(varsigma_m, S, I, R):
    return (alpha0 + alpha1 * (b / (b + I))) * I - (mu + delta) * R

# Define A and B
def A(m, l, psi):
    return (m - l)**psi * (m - l + 1 + psi) - (m - l + 1)**(psi + 1)

def B(m, l, psi):
    return (m - l + 1)**psi * (m - l + 2 + psi) - (m - l)**psi * (m - l + 2 + 2 * psi)

def fractional_sir_solver(zeta, S0, I0, R0):
    S = np.zeros(M)
    I = np.zeros(M)
    R = np.zeros(M)
    S[0] = S0
    I[0] = I0
    R[0] = R0

    for m in range(M - 1):
        sum_S = 0
        sum_I = 0
        sum_R = 0

        for l in range(m + 1):
            if l == 0:
                sum_S += p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
            else:
                sum_S += p1(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)

        S[m + 1] = (w(0) / w(varsigma[m])) * S0 + zeta * p1(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_S
        I[m + 1] = (w(0) / w(varsigma[m])) * I0 + zeta * p2(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_I
        R[m + 1] = (w(0) / w(varsigma[m])) * R0 + zeta * p3(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_R
    return S, I, R

# --- Plotting and PDF Saving ---
pdf_filenames = ["susceptible.pdf", "infected.pdf", "recovered.pdf"]
titles = [
    "Susceptible Population (S) with Different Fractional Orders",
    "Infected Population (I) with Different Fractional Orders",
    "Recovered Population (R) with Different Fractional Orders",
]
ylabels = ["S", "I", "R"]
data_labels = ["S", "I", "R"]

for pdf_filename, title, ylabel, data_label in zip(pdf_filenames, titles, ylabels, data_labels):
    with PdfPages(pdf_filename) as pdf:
      
        # Lists to hold all data
        all_data = []
        
        # Collect data
        for zeta in zeta_values:
          S, I, R = fractional_sir_solver(zeta, S0, I0, R0)
          if data_label == "S":
             all_data.append(S)
          elif data_label == "I":
            all_data.append(I)
          elif data_label =="R":
             all_data.append(R)
      
        # Calculate mins and maxs
        data_min = min([min(data) for data in all_data])
        data_max = max([max(data) for data in all_data])


        # Create a figure
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)
      
        # Plot data
        for zeta, data in zip(zeta_values, all_data):
           ax.plot(varsigma, data, label=f'ζ={zeta}')
        
        ax.set_xlabel('ς')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.set_ylim(data_min, data_max)
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Plots saved to {pdf_filenames}")

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
zeta_values = [0.8, 0.85, 0.9, 0.95]
varsigma_end = 100
p = np.e  # Euler's number
Lambda = 10.25
beta = 0.0004
mu = 0.02
alpha0 = 0.0051
alpha1 = 0.021
b = 0.0083
gamma_val = 0.06
delta = 0.04
h = 0.01
psi = 0.9 #set psi here, should be between 0 and 1

# Time span
varsigma = np.arange(0, varsigma_end, h)
M = len(varsigma)

# Initial conditions
S0 = 100
I0 = 50
R0 = 60

# Define w(varsigma)
def w(varsigma_val):
    return varsigma_val + 1  # Simplify or define appropriately for your model. Setting this to 1 is common.

# Define functions for p_1, p_2, and p_3 (these look correct)
def p1(varsigma_m, S, I, R):
    return Lambda - (2 * beta * S * I) / (S + I + R) - mu * S + delta * R

def p2(varsigma_m, S, I, R):
    return (2 * beta * S * I) / (S + I + R) - (alpha0 + alpha1 * (b / (b + I))) * I - (gamma_val + mu) * I

def p3(varsigma_m, S, I, R):
    return (alpha0 + alpha1 * (b / (b + I))) * I - (mu + delta) * R

# Define A and B
def A(m, l, psi):
    return (m - l)**psi * (m - l + 1 + psi) - (m - l + 1)**(psi + 1)

def B(m, l, psi):
    return (m - l + 1)**psi * (m - l + 2 + psi) - (m - l)**psi * (m - l + 2 + 2 * psi)


def fractional_sir_solver(zeta, S0, I0, R0):
    S = np.zeros(M)
    I = np.zeros(M)
    R = np.zeros(M)
    S[0] = S0
    I[0] = I0
    R[0] = R0
    
    h = 0.1  # Step size

    for m in range(M - 1):
        sum_S = 0.0
        sum_I = 0.0
        sum_R = 0.0

        for l in range(m + 1):
            term1 = 0.0 if (m + 1 - l) == 0 else (m + 1 - l)**(psi - 1)
            term2 = 0.0 if (m - l) == 0 else (m - l)**(psi - 1)
            
            sum_S += (h**psi) * (p1(varsigma[l], S[l], I[l], R[l]) * (term1 - term2))
            sum_I += (h**psi) * (p2(varsigma[l], S[l], I[l], R[l]) * (term1 - term2))
            sum_R += (h**psi) * (p3(varsigma[l], S[l], I[l], R[l]) * (term1 - term2))

        S[m + 1] = S0 + (zeta / gamma(psi + 1)) * sum_S
        I[m + 1] = I0 + (zeta / gamma(psi + 1)) * sum_I
        R[m + 1] = R0 + (zeta / gamma(psi + 1)) * sum_R
    return S, I, R

# --- Plotting and PDF Saving ---
pdf_filenames = ["susceptible.pdf", "infected.pdf", "recovered.pdf"]
titles = [
    "Susceptible Population (S) with Different Fractional Orders",
    "Infected Population (I) with Different Fractional Orders",
    "Recovered Population (R) with Different Fractional Orders",
    ]
ylabels = ["S", "I", "R"]
data_labels = ["S", "I", "R"]

for pdf_filename, title, ylabel, data_label in zip(pdf_filenames, titles, ylabels, data_labels):
    with PdfPages(pdf_filename) as pdf:
      
        # Lists to hold all data
        all_data = []
        
        # Collect data
        for zeta in zeta_values:
          S, I, R = fractional_sir_solver(zeta, S0, I0, R0)
          if data_label == "S":
             all_data.append(S)
          elif data_label == "I":
            all_data.append(I)
          elif data_label =="R":
             all_data.append(R)
      
        # Calculate mins and maxs
        data_min = min([min(data) for data in all_data])
        data_max = max([max(data) for data in all_data])


        # Create a figure
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)
      
        # Plot data
        for zeta, data in zip(zeta_values, all_data):
           ax.plot(varsigma, data, label=f'ζ={zeta}')
        
        ax.set_xlabel('varsigma')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.set_ylim(data_min, data_max)
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Plots saved to {pdf_filenames}")

Plots saved to ['susceptible.pdf', 'infected.pdf', 'recovered.pdf']


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
zeta_values = [0.8, 0.85, 0.9, 0.95]
varsigma_end = 1000
p = np.e
Lambda = 10.25
beta = 0.0004
mu = 0.02
alpha0 = 0.0051
alpha1 = 0.021
b = 0.0083
gamma_val = 0.006
delta = 0.04
h = 0.01
psi = zeta #set psi here

# Time span
varsigma = np.arange(0, varsigma_end, h)
M = len(varsigma)

# Initial conditions
S0 = 100
I0 = 50
R0 = 60

# Define w(varsigma)
def w(varsigma_val):
    return 1  # Correct definition

# Define functions for p_1, p_2, and p_3
def p1(varsigma_m, S, I, R):
    return Lambda - (2 * beta * S * I) / (S + I + R) - mu * S + delta * R

def p2(varsigma_m, S, I, R):
    return (2 * beta * S * I) / (S + I + R) - (alpha0 + alpha1 * (b / (b + I))) * I - (gamma_val + mu) * I

def p3(varsigma_m, S, I, R):
    return (alpha0 + alpha1 * (b / (b + I))) * I - (mu + delta) * R

# Define A and B
def A(m, l, psi):
    return (m - l)**psi * (m - l + 1 + psi) - (m - l + 1)**(psi + 1)

def B(m, l, psi):
    return (m - l + 1)**psi * (m - l + 2 + psi) - (m - l)**psi * (m - l + 2 + 2 * psi)

def fractional_sir_solver(zeta, S0, I0, R0):
    S = np.zeros(M)
    I = np.zeros(M)
    R = np.zeros(M)
    S[0] = S0
    I[0] = I0
    R[0] = R0

    for m in range(M - 1):
        sum_S = 0
        sum_I = 0
        sum_R = 0

        for l in range(m + 1):
            if l == 0:
                sum_S += p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
            else:
                sum_S += p1(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p1(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_I += p2(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p2(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)
                sum_R += p3(varsigma[l - 1], S[l - 1], I[l - 1], R[l - 1]) * A(m, l, psi) + p3(varsigma[l], S[l], I[l], R[l]) * B(m, l, psi)

        S[m + 1] = (w(0) / w(varsigma[m])) * S0 + zeta * p1(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_S
        I[m + 1] = (w(0) / w(varsigma[m])) * I0 + zeta * p2(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_I
        R[m + 1] = (w(0) / w(varsigma[m])) * R0 + zeta * p3(varsigma[m], S[m], I[m], R[m]) + (
                (np.log(p) * zeta * h ** psi) / (gamma(psi + 2) * w(varsigma[m]))) * sum_R
    return S, I, R

# --- Plotting and PDF Saving ---
pdf_filenames = ["susceptible.pdf", "infected.pdf", "recovered.pdf"]
titles = [
    "Susceptible Population (S) with Different Fractional Orders",
    "Infected Population (I) with Different Fractional Orders",
    "Recovered Population (R) with Different Fractional Orders",
]
ylabels = ["S", "I", "R"]
data_labels = ["S", "I", "R"]

for pdf_filename, title, ylabel, data_label in zip(pdf_filenames, titles, ylabels, data_labels):
    with PdfPages(pdf_filename) as pdf:
      
        # Lists to hold all data
        all_data = []
        
        # Collect data
        for zeta in zeta_values:
          S, I, R = fractional_sir_solver(zeta, S0, I0, R0)
          if data_label == "S":
             all_data.append(S)
          elif data_label == "I":
            all_data.append(I)
          elif data_label =="R":
             all_data.append(R)
      
        # Calculate mins and maxs
        data_min = min([min(data) for data in all_data])
        data_max = max([max(data) for data in all_data])


        # Create a figure
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111)
      
        # Plot data
        for zeta, data in zip(zeta_values, all_data):
           ax.plot(varsigma, data, label=f'ζ={zeta}')
        
        ax.set_xlabel('ς')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.set_ylim(data_min, data_max)
        ax.legend()
        ax.grid(True)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)

print(f"Plots saved to {pdf_filenames}")